# Chapter 10 - Artificial Neural Networks

## Perceptron

### Using Scikit-Learn

In [18]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:, (2, 3)]
y = (iris.target == 0).astype(np.int)

per_clf = Perceptron(random_state=42, max_iter=5, tol=None)
per_clf.fit(X, y)

/home/tristan/Miniconda/envs/cs-notebooks/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=5, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=42, shuffle=True, tol=None,
      validation_fraction=0.1, verbose=0, warm_start=False)

In [2]:
y_pred = per_clf.predict([[2, 0.5]])
y_pred

array([1])

### Multi-layer perceptron with TF Estimator API

TensorFlow includes an estimator API that provides pre-baked implementations of various kinds of estimators, as well as supporting the creation of custom estimators. One of the built in estimators is `DNNClassifier`, which can be used to train a multi-layer perceptron.

The following example trains a multi-layer perceptron to classify digits in the MNIST dataset:

In [3]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'details', 'categories', 'url'])

In [4]:
X, y = mnist["data"], mnist["target"]
X.shape

(70000, 784)

In [5]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
X_train = X_train.astype(np.float32).reshape(-1, 28 * 28) / 255.0
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
X_test = X_test.astype(np.float32).reshape(-1, 28 * 28) / 255.0
X_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Convert targets from strings to integers:

In [7]:
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

We can train a multi-layer perceptron using TensorFlow's high-level *TF.Learn* API:

In [10]:
import tensorflow as tf

feature_columns = [tf.feature_column.numeric_column("X", shape=[28 * 28])]
feature_columns

[_NumericColumn(key='X', shape=(784,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [12]:
dnn_clf = tf.estimator.DNNClassifier(
    hidden_units=[300, 100], 
    n_classes=10, 
    feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpfku1hn_2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd28ed101d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_train},
    y=y_train,
    num_epochs=40,
    batch_size=50,
    shuffle=True)

dnn_clf.train(input_fn=input_fn)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpfku1hn_2/model.ckpt.
INFO:tensorflow:loss = 117.7262, step = 0
INFO:tensorflow:global_step/sec: 315.921
INFO:tensorflow:loss = 20.564919, step = 100 (0.318 sec)
INFO:tensorflow:global_step/sec: 353.436
INFO:tensorflow:loss = 11.131307, step = 200 (0.283 sec)
INFO:tensorflow:global_step/sec: 351.801
INFO:tensorflow:loss = 9.044836, step = 300 (0.284 sec)
INFO:tensorflow:global_step/sec: 350.404
INFO:tensorflow:loss = 16.13813, step = 400 (0.285 sec)
INFO:te

INFO:tensorflow:loss = 1.1086681, step = 6900 (0.284 sec)
INFO:tensorflow:global_step/sec: 354.488
INFO:tensorflow:loss = 0.6826881, step = 7000 (0.282 sec)
INFO:tensorflow:global_step/sec: 356.702
INFO:tensorflow:loss = 0.28257573, step = 7100 (0.280 sec)
INFO:tensorflow:global_step/sec: 358.519
INFO:tensorflow:loss = 0.09014869, step = 7200 (0.279 sec)
INFO:tensorflow:global_step/sec: 353.449
INFO:tensorflow:loss = 0.34631354, step = 7300 (0.283 sec)
INFO:tensorflow:global_step/sec: 347.841
INFO:tensorflow:loss = 0.84664774, step = 7400 (0.288 sec)
INFO:tensorflow:global_step/sec: 354.041
INFO:tensorflow:loss = 1.5421405, step = 7500 (0.282 sec)
INFO:tensorflow:global_step/sec: 348.924
INFO:tensorflow:loss = 1.8040048, step = 7600 (0.287 sec)
INFO:tensorflow:global_step/sec: 345.672
INFO:tensorflow:loss = 3.6053228, step = 7700 (0.289 sec)
INFO:tensorflow:global_step/sec: 350.924
INFO:tensorflow:loss = 1.2034683, step = 7800 (0.285 sec)
INFO:tensorflow:global_step/sec: 352.882
INFO:t

INFO:tensorflow:loss = 0.124859884, step = 15100 (0.284 sec)
INFO:tensorflow:global_step/sec: 347.946
INFO:tensorflow:loss = 0.15620138, step = 15200 (0.287 sec)
INFO:tensorflow:global_step/sec: 355.325
INFO:tensorflow:loss = 0.1268428, step = 15300 (0.282 sec)
INFO:tensorflow:global_step/sec: 359.367
INFO:tensorflow:loss = 0.00671531, step = 15400 (0.278 sec)
INFO:tensorflow:global_step/sec: 352.313
INFO:tensorflow:loss = 0.07843885, step = 15500 (0.284 sec)
INFO:tensorflow:global_step/sec: 349.097
INFO:tensorflow:loss = 0.0030956892, step = 15600 (0.287 sec)
INFO:tensorflow:global_step/sec: 355.843
INFO:tensorflow:loss = 0.23368731, step = 15700 (0.281 sec)
INFO:tensorflow:global_step/sec: 349.025
INFO:tensorflow:loss = 0.15551114, step = 15800 (0.287 sec)
INFO:tensorflow:global_step/sec: 357.848
INFO:tensorflow:loss = 0.4372334, step = 15900 (0.279 sec)
INFO:tensorflow:global_step/sec: 350.042
INFO:tensorflow:loss = 0.11553849, step = 16000 (0.286 sec)
INFO:tensorflow:global_step/se

INFO:tensorflow:loss = 0.0120945405, step = 23200 (0.281 sec)
INFO:tensorflow:global_step/sec: 353.306
INFO:tensorflow:loss = 0.09858544, step = 23300 (0.283 sec)
INFO:tensorflow:global_step/sec: 344.251
INFO:tensorflow:loss = 0.05517663, step = 23400 (0.291 sec)
INFO:tensorflow:global_step/sec: 350.29
INFO:tensorflow:loss = 0.084224164, step = 23500 (0.285 sec)
INFO:tensorflow:global_step/sec: 358.878
INFO:tensorflow:loss = 0.025234245, step = 23600 (0.279 sec)
INFO:tensorflow:global_step/sec: 346.112
INFO:tensorflow:loss = 0.051281705, step = 23700 (0.288 sec)
INFO:tensorflow:global_step/sec: 359.645
INFO:tensorflow:loss = 0.4830869, step = 23800 (0.278 sec)
INFO:tensorflow:global_step/sec: 346.076
INFO:tensorflow:loss = 0.03559199, step = 23900 (0.289 sec)
INFO:tensorflow:global_step/sec: 350.561
INFO:tensorflow:loss = 0.054021545, step = 24000 (0.285 sec)
INFO:tensorflow:global_step/sec: 350.663
INFO:tensorflow:loss = 0.045964066, step = 24100 (0.285 sec)
INFO:tensorflow:global_ste

INFO:tensorflow:loss = 0.49556488, step = 31300 (0.289 sec)
INFO:tensorflow:global_step/sec: 353.515
INFO:tensorflow:loss = 0.06009712, step = 31400 (0.283 sec)
INFO:tensorflow:global_step/sec: 352.024
INFO:tensorflow:loss = 0.035602123, step = 31500 (0.284 sec)
INFO:tensorflow:global_step/sec: 355.383
INFO:tensorflow:loss = 0.0030193487, step = 31600 (0.282 sec)
INFO:tensorflow:global_step/sec: 348.292
INFO:tensorflow:loss = 0.031893473, step = 31700 (0.287 sec)
INFO:tensorflow:global_step/sec: 349.672
INFO:tensorflow:loss = 0.023732847, step = 31800 (0.286 sec)
INFO:tensorflow:global_step/sec: 358.17
INFO:tensorflow:loss = 0.08294678, step = 31900 (0.279 sec)
INFO:tensorflow:global_step/sec: 352.86
INFO:tensorflow:loss = 0.12432399, step = 32000 (0.283 sec)
INFO:tensorflow:global_step/sec: 350.33
INFO:tensorflow:loss = 0.045158003, step = 32100 (0.286 sec)
INFO:tensorflow:global_step/sec: 347.304
INFO:tensorflow:loss = 0.005550623, step = 32200 (0.288 sec)
INFO:tensorflow:global_step

INFO:tensorflow:global_step/sec: 345.57
INFO:tensorflow:loss = 0.015084308, step = 39400 (0.289 sec)
INFO:tensorflow:global_step/sec: 355.138
INFO:tensorflow:loss = 0.009892561, step = 39500 (0.281 sec)
INFO:tensorflow:global_step/sec: 353.3
INFO:tensorflow:loss = 0.007989132, step = 39600 (0.283 sec)
INFO:tensorflow:global_step/sec: 360.31
INFO:tensorflow:loss = 0.037904445, step = 39700 (0.278 sec)
INFO:tensorflow:global_step/sec: 346.854
INFO:tensorflow:loss = 0.003251623, step = 39800 (0.288 sec)
INFO:tensorflow:global_step/sec: 358.571
INFO:tensorflow:loss = 0.019610308, step = 39900 (0.279 sec)
INFO:tensorflow:global_step/sec: 349.87
INFO:tensorflow:loss = 0.05230775, step = 40000 (0.286 sec)
INFO:tensorflow:global_step/sec: 352.589
INFO:tensorflow:loss = 0.009111273, step = 40100 (0.284 sec)
INFO:tensorflow:global_step/sec: 347.954
INFO:tensorflow:loss = 0.012778884, step = 40200 (0.288 sec)
INFO:tensorflow:global_step/sec: 349.505
INFO:tensorflow:loss = 0.11860051, step = 40300

INFO:tensorflow:global_step/sec: 351.394
INFO:tensorflow:loss = 0.017170897, step = 47500 (0.285 sec)
INFO:tensorflow:global_step/sec: 351.802
INFO:tensorflow:loss = 0.0029299594, step = 47600 (0.284 sec)
INFO:tensorflow:global_step/sec: 347.072
INFO:tensorflow:loss = 0.029028464, step = 47700 (0.288 sec)
INFO:tensorflow:global_step/sec: 350.709
INFO:tensorflow:loss = 0.0028587282, step = 47800 (0.285 sec)
INFO:tensorflow:global_step/sec: 349.689
INFO:tensorflow:loss = 0.014429746, step = 47900 (0.286 sec)
INFO:tensorflow:Saving checkpoints for 48000 into /tmp/tmpfku1hn_2/model.ckpt.
INFO:tensorflow:Loss for final step: 0.022397973.


In [17]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_test},
    y=y_test,
    shuffle=False)

eval_results = dnn_clf.evaluate(input_fn=test_input_fn)
eval_results

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-07-29-03:23:18
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpfku1hn_2/model.ckpt-48000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-07-29-03:23:18
INFO:tensorflow:Saving dict for global step 48000: accuracy = 0.9817, average_loss = 0.09852992, global_step = 48000, loss = 12.472141
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 48000: /tmp/tmpfku1hn_2/model.ckpt-48000


{'accuracy': 0.9817,
 'average_loss': 0.09852992,
 'loss': 12.472141,
 'global_step': 48000}